In [1]:
import os 
import torch 
import numpy as np 
from matplotlib import pyplot as plt 
from tqdm import tqdm 
im_path = '/mnt/mydisk/zzf/DS-TransUNet-master/vis/0609/img_t/'
target_path = '/mnt/mydisk/zzf/DS-TransUNet-master/vis/0609/target_t/'


In [2]:

R_max = 0
G_max = 0
B_max = 0

for im_pt in tqdm(os.listdir(im_path)):
    
    x = torch.load(im_path+im_pt).cpu().numpy()
    y = torch.load(target_path+im_pt).cpu().numpy()
    R_max = max(np.percentile(x[0,:,:],99),R_max)
    G_max = max(np.percentile(x[1,:,:],99),G_max)
    B_max = max(np.percentile(x[2,:,:],99),B_max)

100%|██████████| 2102/2102 [05:37<00:00,  6.22it/s]


In [4]:
import cv2
res_path = '/mnt/mydisk/zzf/DS-TransUNet-master/vis/0609/res/'
for im_pt in tqdm(os.listdir(im_path)):
    x = torch.load(im_path+im_pt).cpu().numpy()
    y = torch.load(target_path+im_pt).cpu().numpy()
    img = np.zeros((384,384,3)).astype(np.uint8)
    # R_98 = np.percentile(x[0,:,:],99)
    # G_98 = np.percentile(x[1,:,:],99)
    # B_98 = np.percentile(x[2,:,:],99)
    R_98 = R_max
    G_98 = G_max
    B_98 = B_max
    # print(R_98,G_98,B_98)
    if R_98>0:
        img[:,:,0] = (((x[0,:,:]/R_98*1.5))*255).astype(np.uint8)
    else:
        img[:,:,0] = x[0,:,:].astype(np.uint8)
    if G_98>0:  
        img[:,:,1] = (((x[1,:,:]/G_98)*1.5)*255).astype(np.uint8)
    else:
        img[:,:,1] = x[1,:,:].astype(np.uint8)
    if B_98>0:
        img[:,:,2] = (((x[2,:,:]/B_98)*1.5)*255).astype(np.uint8)
    else:
        img[:,:,2] = x[2,:,:].astype(np.uint8)
    cv2.imwrite(res_path+im_pt.replace('.pt','.png'),img)

100%|██████████| 2102/2102 [04:51<00:00,  7.20it/s]


In [10]:
import cv2

cell_dict = {
    "B cells" : 1,
    "CD4 T cells": 2,
    "CD8 T cells": 3,
    "Double-positive T cells" :4,
    "MC1 (HLADR-CD14+CD16-CD11C+CD11b-)" :5,
    "MC2 (HLADR-CD14+CD16-CD11C-CD11b-)" :6,
    "MC3 (HLADR-CD14-CD16-CD11C-CD11b^{hi})" :7,
    "MC4 (HLADR^{hi}CD14^{hi}CD16+CD11c+CD11b+)" :8,
    "MC5 (HLADR+CD14^{hi}CD16+CD11c-CD11b+)" :9,
    "MC6 (HLADR+CD14+CD16-CD11c-CD11b-)" :10,
    "Regulatory T cells" :11,
    "Stroma (Collagen+)" :12,
    "Stroma (FAP+)": 13,
    "Stroma (PDGFRb+)": 14,
    "Stroma (SMA+)": 15,
    "Stroma (Vimentin+)" :16,
    "Tumor (CA9+)" :17,
    "Tumor (Ki67+)" :18,
    "Tumor (VEGF+)" :19,
    "Tumor n.c." :20
}

cell_res = {
    "B cells" : (0,0,255),
    "CD4 T cells": (0,0,255),
    "CD8 T cells": (0,0,255),
    "Double-positive T cells" :(0,0,255),
    "MC1 (HLADR-CD14+CD16-CD11C+CD11b-)" :(0,255,0),
    "MC2 (HLADR-CD14+CD16-CD11C-CD11b-)" :(255,0,0),
    "MC3 (HLADR-CD14-CD16-CD11C-CD11b^{hi})" :(0,255,0),
    "MC4 (HLADR^{hi}CD14^{hi}CD16+CD11c+CD11b+)" :(0,0,255),
    "MC5 (HLADR+CD14^{hi}CD16+CD11c-CD11b+)" :(0,255,0),
    "MC6 (HLADR+CD14+CD16-CD11c-CD11b-)" :(0,255,0),
    "Regulatory T cells" :(0,0,255),
    "Stroma (Collagen+)" :(0,255,0),
    "Stroma (FAP+)": (0,255,0),
    "Stroma (PDGFRb+)": (0,255,0),
    "Stroma (SMA+)": (0,255,0),
    "Stroma (Vimentin+)" :(0,255,0),
    "Tumor (CA9+)" :(0,255,0),
    "Tumor (Ki67+)" :(0,255,0),
    "Tumor (VEGF+)" :(0,255,0),
    "Tumor n.c." :(0,255,0)
}

cell_color_dict = {cell_dict[k]:cell_res[k] for k in cell_dict.keys()}

# cell_color_dict = {
#     1:(255,0,0),
#     2:(0,255,0),
#     3:(0,0,255)
# }

for t_pt in tqdm(os.listdir(target_path)):
    x = torch.argmax(torch.load(target_path+t_pt),dim=0).numpy()
    print(np.unique(x))
    outB = np.zeros((384,384)).astype(np.uint8)
    outG = np.zeros((384,384)).astype(np.uint8)
    outR = np.zeros((384,384)).astype(np.uint8)
    for i in range(1,21):
        mi = (x == i)
        # print(i,cell_color_dict[i])
        outB[mi] = cell_color_dict[i][0]
        outG[mi] = cell_color_dict[i][1]
        outR[mi] = cell_color_dict[i][2]
        # outB[mi] = cell_color_dict[i][0]
        # outG[mi] = cell_color_dict[i][1]
        # outR[mi] = cell_color_dict[i][2]
        
    
    x3 = cv2.merge([outB,outG,outR])
    cv2.imwrite("/mnt/mydisk/zzf/DS-TransUNet-master/vis/0609/target_res/"+t_pt.replace('.pt','.png'),x3)

  0%|          | 0/2102 [00:00<?, ?it/s]

  0%|          | 1/2102 [00:00<07:55,  4.42it/s]

[ 0  1  2  3  4  7 10 11 12 13 15 16 17 18 19 20]


  0%|          | 2/2102 [00:00<07:01,  4.98it/s]

[ 0  1  2  3  4  5  8  9 10 11 12 13 15 16 18 19 20]
[ 0  5  6  8  9 10 12 15 16 17 18 19 20]


...

[0 2 3]
[ 0  2  3  5  6  7  8  9 10 11 12 13 15 16 19 20]


  0%|          | 6/2102 [00:01<06:54,  5.05it/s]

[ 0  3  5  6  8  9 10 12 15 16 20]


  0%|          | 6/2102 [00:01<07:59,  4.37it/s]


KeyboardInterrupt: 

In [16]:
for c in cell_res.keys():
    os.makedirs(os.path.join("/mnt/mydisk/zzf/DS-TransUNet-master/vis/0609/res_by_celltype",c),exist_ok=True)


cell_dict_reverse = {v:k for k,v in cell_dict.items()}
for res_path in tqdm(os.listdir('/mnt/mydisk/zzf/DS-TransUNet-master/vis/0609/res')):
    x = cv2.imread("/mnt/mydisk/zzf/DS-TransUNet-master/vis/0609/res/"+res_path)
    t = torch.load(target_path+res_path.replace('.png','.pt'))
    # print(t.shape)
    m = torch.argmax(t,dim=0).numpy()
    # print(np.unique(m))
    for i in range(1,21):
        x_ = x.copy()
        if np.sum(m==i)==0:
            continue
        mi = ~(m == i)
        x_[:,:,0][mi] = 0 
        x_[:,:,1][mi] = 0 
        x_[:,:,2][mi] = 0 
        cv2.imwrite(os.path.join("/mnt/mydisk/zzf/DS-TransUNet-master/vis/0609/res_by_celltype",cell_dict_reverse[i],res_path),x_)
    

100%|██████████| 2102/2102 [10:28<00:00,  3.34it/s]
